# 瞭解情況

In [ ]:
GL = pd.read_excel('/Users/daniel0522/Downloads/2022 7-11 C2C全.xlsx',sheet_name = '代收款')
GL

In [ ]:
GL.info()
GL.head()
GL['訂單成立日'] = pd.to_datetime(GL['訂單成立日'])
GL['發貨日期'] = pd.to_datetime(GL['發貨日期'])
GL['取貨日期'] = pd.to_datetime(GL['取貨日期'])
GL['取貨日期'] = pd.to_datetime(GL['取貨日期'])

In [ ]:
GL = GL[~GL['訂單號碼'].duplicated(keep = False)]
GL['月份'] = GL['訂單成立日'].dt.month
GL['年份'] = GL['訂單成立日'].dt.year
GL['發貨時長'] = GL['發貨日期']-GL['訂單成立日']
GL

In [ ]:
GL = GL[GL['年份']==2022]

In [ ]:
GL = GL.merge(webside,how = 'inner', left_on = '訂單號碼',right_on = '訂單號碼')
GL

In [ ]:
groupdata = GL.groupby('月份')

In [ ]:
for group_name, group_df in groupdata:
    print(group_df)

In [ ]:
data = {} 
count = []
mean = []
median = []
highest = []
month = []
packday = []
total = []
for group_name, group_df in groupdata:
    count.append(int(group_df['代收金額'].describe()[0]))
    mean.append(round(group_df['代收金額'].describe()[1],2))
    median.append(int(group_df['代收金額'].describe()[5]))
    highest.append(int(group_df['代收金額'].describe()[-1]))
    month.append(group_name)
    packday.append(group_df['發貨時長'].describe()[1])
    total.append(int(group_df['代收金額'].sum()))
data['訂單數'] = count
data['每筆平均金額'] = mean
data['每筆中位數金額'] = median
data['訂單最高金額'] = highest
data['月份'] = month
data['當月平均發貨時長'] = packday
data['總金額'] = total

In [ ]:
df = pd.DataFrame(data)
# 删除原始DataFrame中的"Age"列
df = df.drop(columns=['月份'])

# 将"Age"列插入到第一列
df.insert(0, '月份', data['月份'])

df['當月平均發貨時長'] = df['當月平均發貨時長'].apply(lambda x: x.days)

df

In [ ]:
merge2['月份'] = merge2['訂單成立日'].dt.month
merge2['年份'] = merge2['訂單成立日'].dt.year
merge2['發貨時長'] = merge2['發貨日期']-merge2['訂單成立日']
merge2.drop(columns = ['訂單成立日','付款總金額'],inplace = True)
merge2

In [ ]:
merge2022 = merge2[merge2['年份']==2022]
groupdata = merge2022.groupby('月份')
data = {} 
count = []
mean = []
median = []
highest = []
month = []
packday = []
total = []
for group_name, group_df in groupdata:
    count.append(int(group_df['代收金額'].describe()[0]))
    mean.append(round(group_df['代收金額'].describe()[1],2))
    median.append(int(group_df['代收金額'].describe()[5]))
    highest.append(int(group_df['代收金額'].describe()[-1]))
    month.append(group_name)
    packday.append(group_df['發貨時長'].describe()[1])
    total.append(int(group_df['代收金額'].sum()))
data['訂單數'] = count
data['每筆平均金額'] = mean
data['每筆中位數金額'] = median
data['訂單最高金額'] = highest
data['月份'] = month
data['當月平均發貨時長'] = packday
data['總金額'] = total

df2 = pd.DataFrame(data)
# 删除原始DataFrame中的"Age"列
df2 = df2.drop(columns=['月份'])

# 将"Age"列插入到第一列
df2.insert(0, '月份', data['月份'])

df2['當月平均發貨時長'] = df2['當月平均發貨時長'].apply(lambda x: x.days)

df2

# 爬資訊

In [ ]:
# 載入需要的套件
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import time
from requests.models import REDIRECT_STATI
import pandas as pd

In [ ]:
from selenium import webdriver
browser = webdriver.Chrome('/Users/daniel0522/Downloads/chromedriver-mac-arm64/chromedriver')
browser.get('https://www.instagram.com/GL_tw/')

In [ ]:
soup = BeautifulSoup(browser.page_source, 'html.parser')

In [ ]:
textlist = []
for x in range(1, 40):  # 執行捲軸滾動滾動30次
    browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(10)
    
    soup2 = BeautifulSoup(browser.page_source, 'html.parser')

    for detail in soup2.find_all(class_='_aagv'):
        for i in detail.find_all('img'):
            textlist.append(i)

In [ ]:
textlist

In [ ]:
import re
text = []
for i in range(len(textlist)):
    search = re.compile(r'[0-9]{2}''/''[0-9]{2}')
    c = re.findall(search,str(textlist[i]))
    if '滿額' in str(textlist[i]):
        a = str(textlist[i]).index('滿額')
        text.append(c[0]+str(textlist[i])[a:a+80])
    if '即贈' in str(textlist[i]):
        b = str(textlist[i]).index('滿額')
        text.append(c[0]+str(textlist[i])[b:b+80])

In [ ]:
text = []
for i in range(len(textlist)):
    if '直播' in str(textlist[i]):
        a = str(textlist[i]).index('直播')
        text.append(str(textlist[i])[a-50:a+50])
    if '收單' in str(textlist[i]):
        b = str(textlist[i]).index('收單')
        text.append(str(textlist[i])[b-50:b+50])
    if 'live' in str(textlist[i]):
        b = str(textlist[i]).index('live')
        text.append(str(textlist[i])[b-50:b+50])
    if 'ig' in str(textlist[i]):
        b = str(textlist[i]).index('ig')
        text.append(str(textlist[i])[b-50:b+50])

In [ ]:
df = pd.DataFrame({'訊息': text})
df[:-1]

In [ ]:
df['訊息'].unique()

# 加入直播資訊整理

In [ ]:
launch = ['2022-12-10 12:00:00','2022-11-09 12:00:00','2022-10-12 00:00:00','2022-09-11 12:00:00','2022-08-08 00:00:00','2022-07-09 00:00:00','2022-06-10 00:00:00','2022-05-08 18:00:00','2022-04-09 12:00:00','2022-03-04 18:00:00','2022-02-20 18:00:00','2022-01-01 18:00:00']
start = ['2022-12-09 19:00:00','2022-11-11 19:00:00','2022-10-12 19:00:00','2022-09-11 19:00:00','2022-08-07 19:00:00','2022-07-09 19:00:00','2022-06-10 19:00:00','2022-05-08 19:00:00','2022-04-09 19:00:00','2022-03-04 19:00:00','2022-02-20 19:00:00','2022-01-01 19:00:00']
stop = ['2022-12-14 23:59:00','2022-11-12 23:59:00','2022-10-15 23:59:00','2022-09-15 23:59:00','2022-08-12 23:59:00','2022-07-13 23:59:00','2022-06-14 23:59:00','2022-05-12 23:59:00','2022-04-13 23:59:00','2022-03-08 23:59:00','2022-02-24 23:59:00','2022-01-04 23:59:00']
launch2 = ['2022-12-24 12:00:00','2022-11-26 18:00:00','2022-10-22 18:00:00','nan','2022-08-27 00:00:00','2022-07-23 00:00:00','2022-06-24 00:00:00','2022-05-24 12:00:00','2022-04-23 12:00:00','2022-03-20 18:00:00','nan','nan']
start2 = ['2022-12-23 17:00:00','2022-11-26 19:30:00','2022-10-22 19:30:00','nan','2022-08-26 19:00:00','2022-07-22 19:00:00','2022-06-24 19:00:00','2022-05-24 19:00:00','2022-04-22 19:00:00','2022-03-20 19:00:00','nan','nan']
stop2 = ['2022-12-28 23:59:00','2022-11-30 23:59:00','2022-10-27 23:59:00','nan','2022-08-31 23:59:00','2022-07-27 23:59:00','2022-06-28 23:59:00','2022-05-28 23:59:00','2022-04-27 23:59:00','2022-03-24 23:59:00','nan','nan']
l1target = ['2000','2000','2500','2000','2000','nan','2000','2000','2000','2000','2000','2000']
l1target2 = ['4500','nan','nan','4000','3500','nan','3800','3500','3500','3500','3500','3500']
l1target3 = ['nan','nan','nan','nan','nan','nan','5000','5000','5000','nan','nan','nan']
l2target = ['2500','2000','2500','nan','2000','2000','2000','2000','2000','2000','nan','nan']
l2target2 = ['4500','4500','nan','nan','3500','3500','3800','3500','3500','3500','nan','nan']
l2target3 = ['6000','nan','nan','nan','nan','nan','5000','5000','5000','nan','nan','nan']

In [ ]:
stop = [s.replace("23:59:00", "23:59:59") for s in stop]
stop2 = [s.replace("23:59:00", "23:59:59") for s in stop2]

In [ ]:
start = pd.to_datetime(start)
launch = pd.to_datetime(launch)
stop = pd.to_datetime(stop)
start2 = pd.to_datetime(start2)
launch2 = pd.to_datetime(launch2)
stop2 = pd.to_datetime(stop2)
launch[0] > start[0] #時間越晚越大

In [ ]:
GL = GL[~GL.duplicated(subset=['訂單號碼'], keep=False)]
GL

In [ ]:
live = pd.DataFrame({'上架':launch})
live['直播'] = start
live['收單'] = stop
live['上架2'] = launch2
live['直播2'] = start2
live['收單2'] = stop2
live['直播門檻'] = l1target
live['直播門檻2'] = l1target2
live['直播門檻3'] = l1target3
live['直播2門檻'] = l2target
live['直播2門檻2'] = l2target2
live['直播2門檻3'] = l2target3
live.sort_values('上架',inplace = True)
live.reset_index(drop=True, inplace=True)
live['月份'] = live['上架'].dt.month
live

In [ ]:
GL_new = GL.merge(live, how = 'inner', right_on = '月份', left_on = '月份')
GL_new

# 分類

In [ ]:
for i in range(len(GL_new)):
    if ((GL_new['上架'][i] < GL_new['訂單日期'][i]) & (GL_new['訂單日期'][i] < GL_new['直播'][i])) \
    or ((GL_new['上架2'][i] < GL_new['訂單日期'][i]) & (GL_new['訂單日期'][i] < GL_new['直播2'][i])):
        GL_new.loc[i, '訂單分類'] = str(0)  # 直接買
    elif ((GL_new['直播'][i] < GL_new['訂單日期'][i]) & (GL_new['訂單日期'][i] < GL_new['直播'][i] + timedelta(hours=3))) \
    or ((GL_new['直播2'][i] < GL_new['訂單日期'][i]) & (GL_new['訂單日期'][i] < GL_new['直播2'][i] + timedelta(hours=3))):
        GL_new.loc[i, '訂單分類'] = str(1)  # 直播買
    elif ((GL_new['直播'][i] + timedelta(hours=3) < GL_new['訂單日期'][i]) & (GL_new['訂單日期'][i] < GL_new['收單'][i])) \
    or ((GL_new['直播2'][i] + timedelta(hours=3) < GL_new['訂單日期'][i]) & (GL_new['訂單日期'][i] < GL_new['收單2'][i])):
        GL_new.loc[i, '訂單分類'] = str(2)  # 活動買（不包含前二種）
    else:
        GL_new.loc[i, '訂單分類'] = str(3)  # 其他

In [ ]:
GL_new['直播門檻'].replace('nan', 99999, inplace=True)
GL_new['直播門檻2'].replace('nan', 99999, inplace=True)
GL_new['直播門檻3'].replace('nan', 99999, inplace=True)
GL_new['直播2門檻'].replace('nan', 99999, inplace=True)
GL_new['直播2門檻2'].replace('nan', 99999, inplace=True)
GL_new['直播2門檻3'].replace('nan', 99999, inplace=True)
GL_new

In [ ]:
for i in range(len(GL_new)):
    if  GL_new['付款總金額'][i] >= 1500:
        GL_new.loc[i,'達標訂單'] = '0' #免運費
    if (GL_new['訂單分類'][i] != '3') and (
        ((GL_new['上架'][i] < GL_new['訂單日期'][i]) & (GL_new['訂單日期'][i] < GL_new['直播'][i])) or
        ((GL_new['直播'][i] < GL_new['訂單日期'][i]) & (GL_new['訂單日期'][i] < GL_new['直播'][i] + timedelta(hours=3))) or
        ((GL_new['直播'][i] + timedelta(hours=3) < GL_new['訂單日期'][i]) & (GL_new['訂單日期'][i] < GL_new['收單'][i]))
        ) and (GL_new['付款總金額'][i] >= int(GL_new['直播門檻'][i])):
        GL_new.loc[i,'達標訂單'] = '1' #直播門檻1
    if (GL_new['訂單分類'][i] != '3') and (
        ((GL_new['上架'][i] < GL_new['訂單日期'][i]) & (GL_new['訂單日期'][i] < GL_new['直播'][i])) or
        ((GL_new['直播'][i] < GL_new['訂單日期'][i]) & (GL_new['訂單日期'][i] < GL_new['直播'][i] + timedelta(hours=3))) or
        ((GL_new['直播'][i] + timedelta(hours=3) < GL_new['訂單日期'][i]) & (GL_new['訂單日期'][i] < GL_new['收單'][i]))
        ) and (GL_new['付款總金額'][i] >= int(GL_new['直播門檻2'][i])):
        GL_new.loc[i,'達標訂單'] = '2' #直播門檻2
    if (GL_new['訂單分類'][i] != '3') and (
        ((GL_new['上架'][i] < GL_new['訂單日期'][i]) & (GL_new['訂單日期'][i] < GL_new['直播'][i])) or
        ((GL_new['直播'][i] < GL_new['訂單日期'][i]) & (GL_new['訂單日期'][i] < GL_new['直播'][i] + timedelta(hours=3))) or
        ((GL_new['直播'][i] + timedelta(hours=3) < GL_new['訂單日期'][i]) & (GL_new['訂單日期'][i] < GL_new['收單'][i]))
        ) and (GL_new['付款總金額'][i] >= int(GL_new['直播門檻3'][i])):
        GL_new.loc[i,'達標訂單'] = '3' #直播門檻3
    if (GL_new['訂單分類'][i] != '3') and (
        ((GL_new['上架2'][i] < GL_new['訂單日期'][i]) & (GL_new['訂單日期'][i] < GL_new['直播2'][i])) or
        ((GL_new['直播2'][i] < GL_new['訂單日期'][i]) & (GL_new['訂單日期'][i] < GL_new['直播2'][i] + timedelta(hours=3))) or
        ((GL_new['直播2'][i] + timedelta(hours=3) < GL_new['訂單日期'][i]) & (GL_new['訂單日期'][i] < GL_new['收單2'][i]))
        ) and (GL_new['付款總金額'][i] >= int(GL_new['直播2門檻'][i])):
        GL_new.loc[i,'達標訂單'] = '1' #直播門檻1
    if (GL_new['訂單分類'][i] != '3') and (
        ((GL_new['上架2'][i] < GL_new['訂單日期'][i]) & (GL_new['訂單日期'][i] < GL_new['直播2'][i])) or
        ((GL_new['直播2'][i] < GL_new['訂單日期'][i]) & (GL_new['訂單日期'][i] < GL_new['直播2'][i] + timedelta(hours=3))) or
        ((GL_new['直播2'][i] + timedelta(hours=3) < GL_new['訂單日期'][i]) & (GL_new['訂單日期'][i] < GL_new['收單2'][i]))
        ) and (GL_new['付款總金額'][i] >= int(GL_new['直播2門檻2'][i])):
        GL_new.loc[i,'達標訂單'] = '2' #直播門檻2
    if (GL_new['訂單分類'][i] != '3') and (
        ((GL_new['上架2'][i] < GL_new['訂單日期'][i]) & (GL_new['訂單日期'][i] < GL_new['直播2'][i])) or
        ((GL_new['直播2'][i] < GL_new['訂單日期'][i]) & (GL_new['訂單日期'][i] < GL_new['直播2'][i] + timedelta(hours=3))) or
        ((GL_new['直播2'][i] + timedelta(hours=3) < GL_new['訂單日期'][i]) & (GL_new['訂單日期'][i] < GL_new['收單2'][i]))
        ) and (GL_new['付款總金額'][i] >= int(GL_new['直播2門檻3'][i])):
        GL_new.loc[i,'達標訂單'] = '3' #直播門檻3
    if GL_new['付款總金額'][i] < 1500:
        GL_new.loc[i,'達標訂單'] = '4' #未達標

In [ ]:
a = GL_new[GL_new['訂單號碼'] == '20220221152411880']
a

In [ ]:
GL_new['達標訂單'].unique()

In [ ]:
groupdata = GL_new.groupby('月份')

In [ ]:
from datetime import datetime, time, timedelta
data = {} 
count = []
mean = []
median = []
highest = []
month = []
packday = []
total = []
livecount = []
livemean = []
livemedian = []
livetotal = []
eventcount = []
eventmean = []
eventmedian = []
eventtotal = []
oldcount = []
oldmean = []
oldmedian = []
oldtotal = []
othercount = []
othermean = []
othermedian = []
othertotal = []
lastportion = []

for group_name, group_df in groupdata:
    #上架就買
    a = group_df[(group_df['上架'] < group_df['直播'])&(group_df['上架'] < group_df['訂單日期']) & ( group_df['訂單日期'] < group_df['直播'])|
                (group_df['上架2'] < group_df['直播2'])&(group_df['上架2'] < group_df['訂單日期']) & ( group_df['訂單日期'] < group_df['直播2'])]
    if a.empty:
        oldcount.append(0)
        oldmean.append(0)
        oldmedian.append(0)
        oldtotal.append(0)
    else:
        oldcount.append(int(a['代收金額'].describe()[0]))
        oldmean.append(round(a['代收金額'].describe()[1],2))
        oldmedian.append(int(a['代收金額'].describe()[5]))
        oldtotal.append(int(a['代收金額'].sum()))
        
    b = group_df[
    ((group_df['上架'] < group_df['直播']) & (group_df['直播']+timedelta(hours = 3) < group_df['訂單日期']) & (group_df['訂單日期'] < group_df['收單'])) |
    ((group_df['直播'] < group_df['上架']) & (group_df['上架'] < group_df['訂單日期']) & (group_df['訂單日期'] < group_df['收單'])) |
    ((group_df['上架2'] < group_df['直播2']) & (group_df['直播2']+timedelta(hours = 3) < group_df['訂單日期']) & (group_df['訂單日期'] < group_df['收單2'])) |
    ((group_df['直播2'] < group_df['上架2']) & (group_df['上架2'] < group_df['訂單日期']) & (group_df['訂單日期'] < group_df['收單2']))   
    ]
    eventcount.append(int(b['代收金額'].describe()[0]))
    eventmean.append(round(b['代收金額'].describe()[1],2))
    eventmedian.append(int(b['代收金額'].describe()[5]))
    eventtotal.append(int(b['代收金額'].sum()))
    
    b1 = group_df[
    ((group_df['上架'] < group_df['直播']) & (group_df['直播'] < group_df['訂單日期']) & (group_df['訂單日期'] < group_df['直播']+timedelta(hours = 3))) |
    ((group_df['上架2'] < group_df['直播2']) & (group_df['直播2'] < group_df['訂單日期']) & (group_df['訂單日期'] < group_df['直播2']+timedelta(hours = 3)))
    ]
    if b1.empty:
        livecount.append(0)
        livemean.append(0)
        livemedian.append(0)
        livetotal.append(0)
    else:
        livecount.append(int(b1['代收金額'].describe()[0]))
        livemean.append(round(b1['代收金額'].describe()[1],2))
        livemedian.append(int(b1['代收金額'].describe()[5]))
        livetotal.append(int(b1['代收金額'].sum()))
    
    c = group_df.loc[~group_df.index.isin(a.index) & ~group_df.index.isin(b.index)& ~group_df.index.isin(b1.index)]
    
    othercount.append(int(c['代收金額'].describe()[0]))
    othermean.append(round(c['代收金額'].describe()[1],2))
    othermedian.append(int(c['代收金額'].describe()[5]))
    othertotal.append(int(c['代收金額'].sum()))
                 
    count.append(int(group_df['代收金額'].describe()[0]))
    mean.append(round(group_df['代收金額'].describe()[1],2))
    median.append(int(group_df['代收金額'].describe()[5]))
    highest.append(int(group_df['代收金額'].describe()[-1]))
    month.append(group_name)
    packday.append(group_df['發貨時長'].describe()[1])
    total.append(int(group_df['代收金額'].sum()))
    
    ele = pd.Timestamp('23:00:00').time()
    tw = pd.Timestamp('00:00:00').time()
    one = pd.Timestamp('01:00:00').time()
    group_df['訂單日期'] = group_df['訂單日期'].apply(lambda x: x.replace(year=pd.Timestamp.now().year, month=pd.Timestamp.now().month, day=pd.Timestamp.now().day))

    d = group_df[
    ((group_df['訂單日期'].dt.time >= ele))|((one >= group_df['訂單日期'].dt.time) & (group_df['訂單日期'].dt.time >= tw))]
    lastportion.append(len(d)/len(group_df))
    
data['月份'] = month

data['訂單數'] = count
data['直播訂單數'] = livecount
data['活動訂單數'] = eventcount
data['新品上架直接買訂單數'] = oldcount
data['訂單數(其他)'] = othercount

data['每筆平均金額'] = mean
data['直播每筆平均金額'] = livemean
data['活動每筆平均金額'] = eventmean
data['新品上架直接買每筆平均金額'] = oldmean
data['每筆平均金額(其他)'] = othermean

data['每筆中位數金額'] = median
data['直播每筆中位數金額'] = livemedian
data['活動每筆中位數金額'] = eventmedian
data['新品上架直接買每筆中位數金額'] = oldmedian
data['每筆中位數金額(其他)'] = othermedian

data['總金額'] = total
data['直播總金額'] = livetotal
data['活動總金額'] = eventtotal
data['新品上架直接買總金額'] = oldtotal
data['直接總金額(其他)'] = othertotal

data['訂單最高金額'] = highest
data['當月平均發貨時長'] = packday
data['當月腦波弱下單比例'] = lastportion

In [ ]:
df = pd.DataFrame(data)
# 删除原始DataFrame中的"Age"列
df = df.drop(columns=['月份'])

# 将"Age"列插入到第一列
df.insert(0, '月份', data['月份'])

df['當月平均發貨時長'] = df['當月平均發貨時長'].apply(lambda x: x.days)

df

In [ ]:
df['當月腦波弱下單比例'].describe()

In [ ]:
GL_new.to_excel('GL_new10.xlsx', index=False)